# Tutorial of scDCC on the 10X PBMC CITE data

Note that this tutorial is implemented on Macbook pro 2019 and all code is conducted on CPU. The results reported in the manuscript are conducted on Nvidia GPU P100.

In [1]:
from time import time
import math, os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from scDCC import scDCC
import numpy as np
import collections
from sklearn import metrics
import h5py
import scanpy.api as sc
from preprocess import read_dataset, normalize
from utils import cluster_acc, generate_random_pair_from_proteins, generate_random_pair_from_CD_markers

/home/svcapp/.local/lib/python3.6/site-packages/scanpy/api/__init__.py:6: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning


Read in RNA count matrix (X matrix), normalized protein data (ADT_X matrix) and counts of CD proteins (adt_CD_normalized_counts matrix, which contains counts of CD4, CD8, CD27 and DR).

In [7]:
import anndata as ad

adata_azimuth = ad.read_h5ad("/home/svcapp/tbrain_x/azimuth/pbmc_multimodal_raw.h5ad")
adata_azimuth.obs = ad.read_h5ad("/home/svcapp/tbrain_x/azimuth/pbmc_multimodal.h5ad").obs

/home/svcapp/.local/lib/python3.6/site-packages/anndata/compat/__init__.py:183: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,


In [14]:
adata_azimuth.X.getcol(100).data

array([1., 1., 1., ..., 2., 1., 1.])

In [4]:
adata = sc.AnnData(adata_azimuth)
# print(adata_azimuth.X.shape)
adata.obs

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase,nCount_SCT,nFeature_SCT
L1_AAACCCAAGAAACTCA,7430.0,221,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1,6380.0,2548
L1_AAACCCAAGACATACA,5949.0,211,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1,5693.0,1615
L1_AAACCCACAACTGGTT,6547.0,217,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S,5066.0,1379
L1_AAACCCACACGTACTA,3508.0,207,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1,4984.0,1889
L1_AAACCCACAGCATACT,6318.0,219,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1,5854.0,1620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4089.0,200,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S,8231.0,2200
E2L8_TTTGTTGGTGTGCCTG,6869.0,213,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1,8680.0,2938
E2L8_TTTGTTGGTTAGTTCG,4173.0,208,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S,8555.0,3130
E2L8_TTTGTTGGTTGGCTAT,5979.0,221,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1,8970.0,3533


In [5]:
pwd

'/home/svcapp/userdata/dev/scDCC'

In [7]:
data_mat = h5py.File("data/CITE_PBMC/CITE_CBMC_counts_top2000.h5")
data_mat['ADT_X']

<HDF5 dataset "ADT_X": shape (8617, 10), type "<f8">

In [8]:
data_mat = h5py.File("data/CITE_PBMC/CITE_CBMC_counts_top2000.h5")
x = np.array(data_mat['X'])
y = np.array(data_mat['Y'])
protein_markers = np.array(data_mat['ADT_X'])
data_mat.close()

CD_markers = np.loadtxt("data/CITE_PBMC/adt_CD_normalized_counts.txt", delimiter=',')

# preprocessing scRNA-seq read counts matrix
adata = sc.AnnData(x)
adata.obs['Group'] = y

adata = read_dataset(adata,
                    transpose=False,
                    test_split=False,
                    copy=True)

adata = normalize(adata,
                    size_factors=True,
                    normalize_input=True,
                    logtrans_input=True)

input_size = adata.n_vars

print(adata.X.shape)
print(y.shape)
print(protein_markers.shape)

x_sd = adata.X.std(0)
x_sd_median = np.median(x_sd)
print("median of gene sd: %.5f" % x_sd_median)

### Autoencoder: Successfully preprocessed 2000 genes and 8617 cells.


/home/svcapp/.local/lib/python3.6/site-packages/scanpy/preprocessing/_simple.py:284: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  if isinstance(data, AnnData) and data.isview:
/home/svcapp/.local/lib/python3.6/site-packages/scanpy/utils.py:618: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  if adata.isview:


(8617, 2000)
(8617,)
(8617, 10)
median of gene sd: 0.99994


In [9]:
protein_markers[0]

array([-0.73504908, -0.84318409, -0.0608686 , -0.88142588,  0.13973946,
        1.01112884, -0.2850844 , -0.39783596,  0.19163336,  2.37893217])

In [ ]:
adata.X[0,:100]

array([-0.02786611, -0.10992309, -0.0803274 , -0.29083678, -0.02372558,
       -0.30106473, -0.06937955, -0.05569917, -0.02411963, -0.0182125 ,
       -0.02360963, -0.01819334, -0.02278739, -0.05498387, -0.24285637,
       -0.01974617, -0.06448663, -0.01751066, -0.11605434, -0.01630944,
       -0.14967957, -0.07981861, -0.02766259, -0.05851728, -0.01508371,
       -0.02685211, -0.06482514, -0.24545565, -0.20200197, -0.0227434 ,
       -0.10728859, -0.11516444, -0.02180206,  0.3963647 , -0.16631743,
       -0.36631155, -0.11644471, -0.08903953, -0.13577938, -0.02443773,
       -0.05472139, -0.30708236, -0.13862017, -0.3932852 , -0.21410878,
       -0.05726252, -0.04280129, -0.2520631 , -0.13377403, -0.21855088,
       -0.13948585, -0.04075027, -0.16554175, -0.05139533, -0.33938122,
       -0.07742169, -0.08160498, -0.14906165,  0.1974773 , -0.19126202,
       -0.2682153 , -0.05666018, -0.44680244, -0.19435206, -0.33974645,
       -0.01901805, -0.14474413, -0.07750179, -0.05698354, -0.30

Generate constraints based on all proteins and CDs. The details are described here (the section "Constraint Construction" in the manuscript):

1. We generated 20,000 ("n_pairwise_1") constraints based on all protein levels. We calculated Euclidean distances for all possible pairs of cells based on the normalized protein data and chose the 0.5th and 95th percentile of all pairwise distances as the must-link and cannot-link constraint cutoffs. Thirdly, we repeatedly sampled two cells and if the Euclidean distance between the two cells was less than the 0.5th percentile of all pairwise distances, we defined it as a must-link constraint; if the Euclidean distance between the two cells was greater than the 95th percentile of all pairwise distances, we defined it as a cannot-link constraint.

2. To separate CD4 and CD8 T cells, we further added 5000 ("n_pairwise_2") constraints based on following rules: if one cell has high CD4 protein level ( > 70th percentile) and low CD8 protein level (< 30th percentile), and another cell has high CD8 protein level ( > 70th percentile) and low CD4 protein level (< 30th percentile), then a cannot-link is constructed. To further identify subtypes of CD4 and CD8 T cells (CD8+CD27-, CD8+CD27+, CD4+CD27+, CD4+CD27-DR+, CD4+CD27-DR-), we generate must-links for each subtype. Taking the CD8+CD27+ T cells as an example, we require that the two randomly selected cells to form a must-link constraint should have both high CD8 protein levels (> 85th percentile) and high CD27 protein levels (> 85th percentile). In contrast, the two cells to form a must-link constraint for the CD8+CD27- subtype should have high CD8 protein levels (> 85th percentile) but low CD27 protein levels (< 30th percentile). For CD4+CD27+, CD4+CD27-DR+, CD4+CD27-DR- cells, we applied similar rules to construct must-links.

In [9]:
n_pairwise_1 = 20000
ml_ind1_1, ml_ind2_1, cl_ind1_1, cl_ind2_1 = generate_random_pair_from_proteins(protein_markers, n_pairwise_1, ML=0.005, CL=0.95)

print("Must link paris: %d" % ml_ind1_1.shape[0])
print("Cannot link paris: %d" % cl_ind1_1.shape[0])

n_pairwise_2 = 5000
ml_ind1_2, ml_ind2_2, cl_ind1_2, cl_ind2_2 = generate_random_pair_from_CD_markers(CD_markers, n_pairwise_2, low1=0.3, high1=0.7, low2=0.3, high2=0.85)

print("Must link paris: %d" % ml_ind1_2.shape[0])
print("Cannot link paris: %d" % cl_ind1_2.shape[0])

ml_ind1 = np.append(ml_ind1_1, ml_ind1_2)
ml_ind2 = np.append(ml_ind2_1, ml_ind2_2)
cl_ind1 = np.append(cl_ind1_1, cl_ind1_2)
cl_ind2 = np.append(cl_ind2_1, cl_ind2_2)

Must link paris: 1817
Cannot link paris: 18183
Must link paris: 440
Cannot link paris: 4560


In [6]:
ml_ind1.shape

(2118,)

Construct the model and pretrain the ZINB autoencoder for 300 epochs. Here we load the pretrained weights.

In [11]:
sd = 2.5
gamma = 1.0

model = scDCC(input_dim=adata.n_vars, z_dim=32, n_clusters=12, 
            encodeLayer=[256, 64], decodeLayer=[64, 256], sigma=sd, gamma=gamma,
            ml_weight=1., cl_weight=1., save_dir="results/cite_pbmc/").cpu()

model.pretrain_autoencoder(x=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, 
                               batch_size=256, epochs=300, ae_weights="CITE_PBMC_AE_weights.pth.tar")


print("==> loading checkpoint '{}'".format("CITE_PBMC_AE_weights.pth.tar"))
checkpoint = torch.load("pretrained_weights/CITE_PBMC_AE_weights.pth.tar")
model.load_state_dict(checkpoint['ae_state_dict'])

Pretraining stage
Pretrain epoch [1/1], ZINB loss:0.5030
Pretrain epoch [2/1], ZINB loss:0.5061
Pretrain epoch [3/1], ZINB loss:0.4862
Pretrain epoch [4/1], ZINB loss:0.4969
Pretrain epoch [5/1], ZINB loss:0.4638
Pretrain epoch [6/1], ZINB loss:0.4498
Pretrain epoch [7/1], ZINB loss:0.4499
Pretrain epoch [8/1], ZINB loss:0.4310
Pretrain epoch [9/1], ZINB loss:0.3987
Pretrain epoch [10/1], ZINB loss:0.3808
Pretrain epoch [11/1], ZINB loss:0.3486
Pretrain epoch [12/1], ZINB loss:0.3491
Pretrain epoch [13/1], ZINB loss:0.3388
Pretrain epoch [14/1], ZINB loss:0.3494
Pretrain epoch [15/1], ZINB loss:0.3415
Pretrain epoch [16/1], ZINB loss:0.3244
Pretrain epoch [17/1], ZINB loss:0.3399
Pretrain epoch [18/1], ZINB loss:0.3173
Pretrain epoch [19/1], ZINB loss:0.3147
Pretrain epoch [20/1], ZINB loss:0.3077
Pretrain epoch [21/1], ZINB loss:0.3107
Pretrain epoch [22/1], ZINB loss:0.3228
Pretrain epoch [23/1], ZINB loss:0.3069
Pretrain epoch [24/1], ZINB loss:0.3104
Pretrain epoch [25/1], ZINB los

<All keys matched successfully>

Clustering with constraints.

In [12]:
if not os.path.exists("results"):
            os.makedirs("results")

y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, sf=adata.obs.size_factors, y=y, batch_size=256, num_epochs=2000, 
            ml_ind1=ml_ind1, ml_ind2=ml_ind2, cl_ind1=cl_ind1, cl_ind2=cl_ind2,
            update_interval=1, tol=0.001, save_dir="results/cite_pbmc")

TypeError: fit() got an unexpected keyword argument 'ml_ind1'

In [13]:
import sklearn.utils.linear_assignment_

ModuleNotFoundError: No module named 'sklearn.utils.linear_assignment_'